In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os, sys, argparse
import tensorflow.keras.backend as K
#from tensorflow.keras.utils import multi_gpu_model
from tensorflow.keras.callbacks import TensorBoard, TerminateOnNaN

from hourglass.model import get_hourglass_model
from hourglass.data import hourglass_dataset
from hourglass.loss import get_loss
from hourglass.callbacks import EvalCallBack, CheckpointCleanCallBack
from common.utils import get_classes, get_matchpoints, get_model_type, optimize_tf_gpu
from common.model_utils import get_optimizer
from hourglass.postprocess import post_process_heatmap_simple
from tensorflow.keras.models import load_model
from eval import keypoint_accuracy
from common.model_utils import get_normalize
from tqdm import tqdm
# Try to enable Auto Mixed Precision on TF 2.0
# os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
# os.environ['TF_AUTO_MIXED_PRECISION_GRAPH_REWRITE_IGNORE_PERFORMANCE'] = '1'
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
# optimize_tf_gpu(tf, K)

In [2]:
# Arguments parsed from command line can be set here
# Model definition options:
num_stacks=2
mobile=False
tiny=False
model_input_shape="256x256"
weights_path=None

# Data options
dataset_path="data/mpii"
classes_path="configs/mpii_classes.txt"
matchpoint_path="configs/mpii_match_point.txt"

# Training options
batch_size=8
optimizer="RMSProp"
loss_type="mse"
learning_rate=5e-4
decay_type=None
mixed_precision=False
init_epoch=0
total_epoch=100
gpu_num=1

height, width = model_input_shape.split('x')
model_input_shape = (int(height), int(width)) 
orig_img_shape = (1280, 720)    # Height and width in pixels of input images, can be read from file or manually specified

In [3]:
# def main(args):
log_dir = 'logs/000'
os.makedirs(log_dir, exist_ok=True)

class_names = get_classes(classes_path)
num_classes = len(class_names)
if matchpoint_path:
    matchpoints = get_matchpoints(matchpoint_path)
else:
    matchpoints = None

# choose model type
if tiny:
    num_channels = 128
else:
    num_channels = 256

if mixed_precision:
    tf_major_version = float(tf.__version__[:3])
    if tf_major_version >= 2.1:
        # apply mixed_precision for valid TF version
        from tensorflow.keras.mixed_precision import experimental as mixed_precision

        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_policy(policy)
    else:
        raise ValueError('Tensorflow {} does not support mixed precision'.format(tf.__version__))


In [4]:
output_shape = (int(model_input_shape[0]/4), int(model_input_shape[1]/4))

In [5]:
from common.data_utils import random_horizontal_flip, random_vertical_flip, random_brightness
from common.data_utils import random_grayscale, random_chroma, random_contrast, random_sharpness, random_blur, random_histeq, random_rotate_angle
from common.data_utils import crop_single_object, rotate_single_object, crop_image, normalize_image, transform_keypoints, generate_gt_heatmap
from PIL import Image


def map_dataset_to_image_heatmaps_val(imagefile, center, scale, keypoints):
            
    img = tf.io.read_file(imagefile)
    decoded_img = tf.io.decode_png(img, channels=3)
#     orig_img_shape = decoded_img.shape
    resized_img = tf.image.resize(decoded_img, model_input_shape)
    image = resized_img
    
#     image = tf.expand_dims(resized_img, axis=0)


#     img = tf.io.read_file(imagefile)
#     if img.mode != 'RGB':
#         img = img.convert('RGB')
#     image = np.array(img)
#     img.close()
    
#     image_shape = image.shape

    rotate_angle = 0
#     image = crop_image(image, center, scale, model_input_shape, rotate_angle)
    
    # transform keypoints to cropped image reference
#     transformed_keypoints = transform_keypoints(keypoints, center, scale, output_shape, rotate_angle)

        # in case we got an empty image, bypass the sample
#     if image is None:
#         return None, None, None
    
    # normalize image
#     image = normalize_image(image, self.get_color_mean())



    # Data Augmentation
#     image, keypoints = crop_single_object(image, keypoints, center, scale, model_input_shape)
#     seed = tf.random.uniform(shape=[2], maxval=3, dtype=tf.int32)
#     image = tf.image.stateless_random_brightness(image, max_delta=0.95, seed=seed)
#     image = tf.image.stateless_random_contrast(image, lower=0.1, upper=0.9, seed=seed)
#     image = tf.image.stateless_random_hue(image, 0.2, seed)
#     image = tf.image.stateless_random_jpeg_quality(image, 75, 95, seed)
#     image = tf.image.stateless_random_saturation(image, lower=0.5, upper=1.0, seed=seed)


    # Rescale keypoints from original_img_shape to output_shape
    resized_scale = tf.divide(orig_img_shape, output_shape)
    resized_scale = tf.concat([resized_scale, [1]], 0)  # adding third dimension for visibility dimension in the keypoints 
    keypoints = tf.multiply(keypoints, resized_scale)

    # generate ground truth keypoint heatmap
#     gt_heatmap = generate_gt_heatmap(keypoints, output_shape)

#     out_heatmaps = []
#     for m in range(num_stacks):
#         out_heatmaps.append(gt_heatmap)
            
#     return (image, tf.stack(out_heatmaps, axis=-1))

    return (image, keypoints)




In [6]:
import numpy as np
import json

def dataset_from_annotations(annotations, image_path, validation_set=False):

    image_filenames = []
    centers = []
    keypoints = []
    scales = []
    
    for annotation in annotations:
        image_filename = os.path.join(image_path, annotation['img_paths'])
        center = np.array(annotation['objpos'])
        keypoint = np.array(annotation['joint_self'])
        scale = annotation['scale_provided']
        
        # adjust center/scale slightly to avoid cropping limbs

        if center[0] != -1:
            center[1] = center[1] + 15 * scale
            scale = scale * 1.25
        
        if annotation['isValidation'] == validation_set:        
            image_filenames.append(image_filename)
            centers.append(center)
            keypoints.append(keypoint)
            scales.append(scale)
        else:
            pass
    img_filenames = tf.convert_to_tensor(image_filenames)
    img_centers = tf.convert_to_tensor(centers)
    img_scales = tf.convert_to_tensor(scales)
    img_keypoints = tf.convert_to_tensor(keypoints)
    return image_filenames, centers, scales, keypoints
  
                 
            
    
        
# dataset with tf.data.Dataset and training using model.fit()
json_file = "data/mpii/annotations.json"
image_path = "data/mpii/images/"
with open(json_file) as f:
    annotations = json.load(f)
    
    
    
    

In [7]:
img_filenames, img_centers, img_scales, img_keypoints = dataset_from_annotations(annotations, image_path,validation_set=True)      
tfdataset_val= tf.data.Dataset.from_tensor_slices((img_filenames, img_centers, img_scales, img_keypoints))


In [8]:
AUTOTUNE = tf.data.AUTOTUNE
tfdataset_mapped_val = tfdataset_val.map(map_dataset_to_image_heatmaps_val, num_parallel_calls=AUTOTUNE).batch(batch_size, drop_remainder=True).prefetch(AUTOTUNE)

In [9]:
model_path = "logs/000/ep024-loss0.001-val_acc0.725.h5"
model = load_model(model_path, compile=False)

In [10]:
tfdataset_mapped_val
data_item = next(iter(tfdataset_mapped_val))
data_item[1].shape

TensorShape([8, 16, 3])

In [11]:
tfdataset_mapped_val.element_spec[0].shape[0]

8

In [12]:
prediction = model.predict(data_item[0])
prediction[1].shape

(8, 64, 64, 16)

In [13]:
def eval_callback_new(model, tfdataset_mapped_val, batch_size, class_names, model_input_shape, threshold):
    succeed_dict = {class_name: 0 for class_name in class_names}
    fail_dict = {class_name: 0 for class_name in class_names}
    accuracy_dict = {class_name: 0. for class_name in class_names}

    normalize = get_normalize(model_input_shape)
#     threshold = 0.5

    pbar = tqdm(total=len(tfdataset_mapped_val), desc='Eval model')
    for batch in iter(tfdataset_mapped_val):
        prediction = model.predict_on_batch(batch[0])
        pred_keypoints= tf.TensorArray(tf.double, size=batch_size)
        for i in range(0,batch_size):
            pred_keypoints = pred_keypoints.write(i,post_process_heatmap_simple(prediction[1][i]))
            result_list = keypoint_accuracy(pred_keypoints.stack()[i], batch[1][i], threshold, normalize)
            for i, class_name in enumerate(class_names):
                if result_list[i] == 0:
                    fail_dict[class_name] = fail_dict[class_name] + 1
                elif result_list[i] == 1:
                    succeed_dict[class_name] = succeed_dict[class_name] + 1


        pbar.update(1)
    pbar.close()

    for i, class_name in enumerate(class_names):
        accuracy_dict[class_name] = succeed_dict[class_name] * 1.0 / (succeed_dict[class_name] + fail_dict[class_name])

    total_succeed = np.sum(list(succeed_dict.values()))
    total_fail = np.sum(list(fail_dict.values()))
    total_accuracy = total_succeed * 1.0 / (total_fail + total_succeed)
    
    return total_accuracy


In [ ]:
eval_callback_new(tfdataset_mapped_val, batch_size, class_names, model_input_shape, threshold)

In [ ]:
succeed_dict = {class_name: 0 for class_name in class_names}
fail_dict = {class_name: 0 for class_name in class_names}
accuracy_dict = {class_name: 0. for class_name in class_names}

normalize = get_normalize(model_input_shape)
threshold = 0.5

pbar = tqdm(total=47, desc='Eval model')
for batch in iter(tfdataset_mapped_val):
    prediction = model.predict_on_batch(batch[0])
    pred_keypoints= tf.TensorArray(tf.double, size=batch_size)
    for i in range(0,batch_size):
        pred_keypoints = pred_keypoints.write(i,post_process_heatmap_simple(prediction[1][i]))
        result_list = keypoint_accuracy(pred_keypoints.stack()[i], batch[1][i], threshold, normalize)
        for i, class_name in enumerate(class_names):
            if result_list[i] == 0:
                fail_dict[class_name] = fail_dict[class_name] + 1
            elif result_list[i] == 1:
                succeed_dict[class_name] = succeed_dict[class_name] + 1

        
    pbar.update(1)
pbar.close()

for i, class_name in enumerate(class_names):
    accuracy_dict[class_name] = succeed_dict[class_name] * 1.0 / (succeed_dict[class_name] + fail_dict[class_name])

total_succeed = np.sum(list(succeed_dict.values()))
total_fail = np.sum(list(fail_dict.values()))
total_accuracy = total_succeed * 1.0 / (total_fail + total_succeed)


In [15]:
i=6
pred_keypoints = tf.TensorArray(tf.double, size=batch_size)
pred_keypoints = pred_keypoints.write(i, post_process_heatmap_simple(prediction[1][i], 0.001))
pred_keypoints.stack()[4].shape

TensorShape([16, 3])

In [21]:
batch = next(iter(tfdataset_mapped_val))
prediction = model.predict_on_batch(batch[0])
prediction[1][0].shape


(64, 64, 16)

In [ ]:
pred_keypoints = post_process_heatmap_simple(prediction[1][:,:,1,:], 0.1)
tf.stack(pred_keypoints).shape

In [ ]:
data_item[1][60].shape

In [ ]:
result_list = keypoint_accuracy(pred_keypoints.stack()[0], data_item[1][0], 0.5, normalize)

In [ ]:
succeed_dict = {class_name: 0 for class_name in class_names}
fail_dict = {class_name: 0 for class_name in class_names}
accuracy_dict = {class_name: 0. for class_name in class_names}


for i, class_name in enumerate(class_names):
    if result_list[i] == 0:
        fail_dict[class_name] = fail_dict[class_name] + 1
    elif result_list[i] == 1:
        succeed_dict[class_name] = succeed_dict[class_name] + 1

for i, class_name in enumerate(class_names):
    accuracy_dict[class_name] = succeed_dict[class_name] * 1.0 / (succeed_dict[class_name] + fail_dict[class_name])


In [ ]:
total_succeed = np.sum(list(succeed_dict.values()))
total_fail = np.sum(list(fail_dict.values()))
total_accuracy = total_succeed * 1.0 / (total_fail + total_succeed)


In [ ]:
total_accuracy